In [8]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load dataset
df = pd.read_csv("twitter_practical(4A).csv")

# Rename columns
df.rename(columns={"label": "category", "tweet": "clean_text"}, inplace=True)

# Drop missing values
df.dropna(subset=['category', 'clean_text'], inplace=True)

# Binary label mapping: 0 → Hate Speech, 1 or 2 → Not Hate Speech
df['category'] = df['category'].map({0: 1, 1: 0, 2: 0})  # 1 = Hate Speech, 0 = Not Hate Speech

# Text cleaning function
def clean_text(text):
    if isinstance(text, float):
        return ""
    text = re.sub(r"http\S+", "", text)        # Remove URLs
    text = re.sub(r"@\w+", "", text)           # Remove mentions
    text = re.sub(r"#\w+", "", text)           # Remove hashtags
    text = re.sub(r"[^\w\s]", "", text)        # Remove punctuation
    text = text.lower()                        # Convert to lowercase
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Clean the text
df['clean_text'] = df['clean_text'].astype(str).apply(clean_text)

# Vectorization
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')
X = vectorizer.fit_transform(df['clean_text'])
y = df['category']

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
weights_dict = dict(zip(np.unique(y), class_weights))

# Train Logistic Regression model
model = LogisticRegression(max_iter=500, class_weight=weights_dict)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["Not Hate Speech", "Hate Speech"]))

# Prediction function
def predict_tweet(tweet):
    tweet_cleaned = clean_text(tweet)
    tweet_vectorized = vectorizer.transform([tweet_cleaned])
    prediction = model.predict(tweet_vectorized)[0]
    label_map = {1: "Not Hate Speech", 0: "Hate Speech"}
    return label_map[prediction]

# Sample tweets
sample_tweets = [
    "I hate this so much! This is terrible!",
    "Wow, this is amazing! Great job!",
    "You are so dumb, I can't believe this!",
    "Wishing everyone a happy and peaceful day!",
    "This is absolutely disgusting and unacceptable!"
]

# Make predictions
for tweet in sample_tweets:
    print(f"Tweet: {tweet}\nPrediction: {predict_tweet(tweet)}\n")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anjali\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Accuracy: 0.8937900829031753
                 precision    recall  f1-score   support

Not Hate Speech       0.37      0.71      0.48       448
    Hate Speech       0.98      0.91      0.94      5945

       accuracy                           0.89      6393
      macro avg       0.67      0.81      0.71      6393
   weighted avg       0.93      0.89      0.91      6393

Tweet: I hate this so much! This is terrible!
Prediction: Hate Speech

Tweet: Wow, this is amazing! Great job!
Prediction: Not Hate Speech

Tweet: You are so dumb, I can't believe this!
Prediction: Hate Speech

Tweet: Wishing everyone a happy and peaceful day!
Prediction: Not Hate Speech

Tweet: This is absolutely disgusting and unacceptable!
Prediction: Hate Speech



In [4]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace, when, rand
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

# Start Spark session
spark = SparkSession.builder.appName("FixedSentimentAnalysis").getOrCreate()
start_time = time.time()

# Load dataset
df = spark.read.csv("twitter.csv", header=True, inferSchema=True).select("tweet", "label")

# Clean tweets
df_clean = df.withColumn("tweet", lower(col("tweet")))
df_clean = df_clean.withColumn("tweet", regexp_replace("tweet", r"http\S+|www\S+", ""))
df_clean = df_clean.withColumn("tweet", regexp_replace("tweet", r"@\w+", ""))
df_clean = df_clean.withColumn("tweet", regexp_replace("tweet", r"#", ""))
df_clean = df_clean.withColumn("tweet", regexp_replace("tweet", r"[^\w\s]", ""))
df_clean = df_clean.withColumn("tweet", regexp_replace("tweet", r"\d+", ""))

# Handle class imbalance
positive = df_clean.filter(col("label") == 1)
negative = df_clean.filter(col("label") == 0)
neg_sample = negative.sample(False, positive.count() / negative.count(), seed=42)
df_train = neg_sample.union(positive).orderBy(rand())

# Test sentences
test_sentences = [
    ("I hate this product! It's the worst!",),
    ("This is amazing! I love it.",),
    ("Not bad, but could be better.",),
    ("Absolutely terrible experience.",),
    ("Had a fantastic time using this app.",),
]
df_test = spark.createDataFrame(test_sentences, ["tweet"])

# Clean test tweets
for pattern in [r"http\S+|www\S+", r"@\w+", r"#", r"[^\w\s]", r"\d+"]:
    df_test = df_test.withColumn("tweet", regexp_replace("tweet", pattern, ""))
df_test = df_test.withColumn("tweet", lower(col("tweet")))

# ML Pipeline
tokenizer = RegexTokenizer(inputCol="tweet", outputCol="words", pattern="\\W")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
vectorizer = CountVectorizer(inputCol="filtered_words", outputCol="raw_features", vocabSize=10000)
idf = IDF(inputCol="raw_features", outputCol="features")
lr = LogisticRegression(featuresCol="features", labelCol="label")  # <-- use original label

pipeline = Pipeline(stages=[tokenizer, remover, vectorizer, idf, lr])

# Train model
model = pipeline.fit(df_train)

# Predict
predictions = model.transform(df_test)

# Map prediction
predictions = predictions.withColumn(
    "sentiment",
    when(col("prediction") == 1.0, "Positive").otherwise("Negative")
)

print(f"\nExecution Time: {time.time() - start_time:.2f} seconds")
predictions.select("tweet", "prediction", "sentiment").show(truncate=False)

# Stop Spark
spark.stop()



Execution Time: 6.87 seconds


Py4JJavaError: An error occurred while calling o894.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 173.0 failed 1 times, most recent failure: Lost task 0.0 in stage 173.0 (TID 132) (LAPTOP-MGPUBFRL executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCh